In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import pymysql

In [3]:
def starbucks_store(sido, gugun):
    result = []

    # 크롬으로 사이트 열기
    driver = webdriver.Chrome()
    driver.get("https://www.starbucks.co.kr/store/store_map.do")
    driver.implicitly_wait(10)
    driver.find_element(By.LINK_TEXT, "지역 검색").click()
    driver.implicitly_wait(10)
    try:
        driver.find_element(By.LINK_TEXT, sido).click()
    except:
        print("시도 입력 오류")
    driver.implicitly_wait(10)
    try:
        driver.find_element(By.LINK_TEXT, gugun).click()
    except:
        print("구군 입력 오류")
    driver.implicitly_wait(60)

    # 크롤링
    driver.execute_script("document.querySelector('#mCSB_3').style.cssText = 'overflow: visible;'")
    ul_tag = driver.find_element(By.CLASS_NAME, "quickSearchResultBoxSidoGugun")
    store_list = ul_tag.find_elements(By.TAG_NAME, "li")
    for store in store_list:
        name = store.get_attribute("data-name")
        addressAndTel = store.find_element(By.CSS_SELECTOR, "p").text.split("\n")
        address = addressAndTel[0]
        tel = addressAndTel[1]
        latitude = store.get_attribute("data-lat")
        longtitude = store.get_attribute("data-long")
        result.append((name, address, tel, latitude, longtitude))

    driver.quit()

    return result


In [4]:
store_list = starbucks_store("부산", "전체")

df = pd.DataFrame(store_list, columns=["점포명", "주소", "전화번호", "위도", "경도"])
df.to_csv("../data/스타벅스 실습.csv", encoding="utf8")

In [5]:
conn = pymysql.connect(
    host="localhost",
    user="jhm",
    password="1234",
    database="pydb",
    charset="utf8"
)
c = conn.cursor()
c.executemany(
    "INSERT INTO starbucks_store(name, address, tel, latitude, longtitude) VALUES(%s, %s, %s, %s, %s)",
    store_list
)
conn.commit()
conn.close()